In [7]:
# 1. 패키지 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
def load_data():
    승하차_파일 = "../../data/결과/승하차/통합/2호선_승하차인원_통합.csv"
    혼잡도_파일 = "../../data/결과/혼잡도/통합/2호선_혼잡도_통합.csv"
    승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
    혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
    # 시간컬럼명 0패딩 통일
    def fix_timecols(cols):
        return [c if ':' not in c else c.zfill(5) for c in cols]
    승하차_df.columns = fix_timecols(승하차_df.columns)
    혼잡도_df.columns = fix_timecols(혼잡도_df.columns)
    return 승하차_df, 혼잡도_df


In [9]:
# 3. 승하차 데이터를 상행/하행으로 2배 복제
def duplicate_for_directions(승하차):
    dfs = []
    for direction in ["상행", "하행"]:
        tmp = 승하차.copy()
        tmp['방향'] = direction
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)


In [ ]:
# 전처리 함수
def standardize_columns(df, do_direction=True):
    df["평일주말"] = df["평일주말"].astype(str).str.strip()
    df["구분"] = df["구분"].astype(str).str.strip()
    df["역번호"] = df["역번호"].astype(str).str.strip()
    if do_direction and "방향" in df.columns:
        df["방향"] = df["방향"].astype(str).str.strip()
    return df




> 학습데이터 생성: 13016520행, 실패 131480 / 총시도 13148000


In [11]:
# 5. AI 딥러닝 학습 데이터 생성
def build_dl_dataset(승하차, 혼잡도, 정원=2000):
    rows = []
    시간컬럼들 = [col for col in 승하차.columns if ':' in col]
    fail, total = 0, 0
    for idx, row in 승하차.iterrows():
        for col in 시간컬럼들:
            total += 1
            hour = int(col.split(":")[0])
            평일주말 = 1 if row["평일주말"] == "주말" else 0
            상행 = 1 if row["방향"] == "상행" else 0
            승차 = 1 if row["구분"] == "승차" else 0
            matched = 혼잡도[
                (혼잡도["역번호"].astype(str).str.strip() == row["역번호"]) &
                (혼잡도["평일주말"].astype(str).str.strip() == row["평일주말"]) &
                (혼잡도["구분"].astype(str).str.strip() == row["방향"])
            ]
            if matched.empty or col not in 혼잡도.columns:
                fail += 1
                continue
            try:
                congestion = float(matched.iloc[0][col])
                승하차인원 = float(row[col])
            except Exception as e:
                fail += 1
                continue
            y = int(congestion * 정원 / 100)
            rows.append([
                int(row["역번호"]), hour, 평일주말, 상행, 승차, 승하차인원, congestion, y
            ])
    print(f"> 학습데이터 생성: {len(rows)}행, 실패 {fail} / 총시도 {total}")
    cols = ["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도", "target"]
    return pd.DataFrame(rows, columns=cols)

In [12]:
# 6. 전체 파이프라인 (학습+예측)
승하차, 혼잡도 = load_data()
승하차_expanded = duplicate_for_directions(승하차)
승하차_expanded = standardize_columns(승하차_expanded)
혼잡도 = standardize_columns(혼잡도)
df = build_dl_dataset(승하차_expanded, 혼잡도)
print(df.head())
if df.shape[0]==0:
    raise ValueError("학습데이터 생성 실패! 컬럼 또는 값 일치 여부 확인 필요.")


> 학습데이터 생성: 13016520행, 실패 131480 / 총시도 13148000
   역번호  hour  평일주말  상행  승차  승하차인원   혼잡도  target
0  201     5     1   1   1   39.0   5.2     104
1  201     6     1   1   1   51.0   9.2     184
2  201     7     1   1   1   95.0   9.9     198
3  201     8     1   1   1  104.0  18.0     360
4  201     9     1   1   1  129.0  21.0     420


In [13]:
# 7. 입력/정규화/트레인테스트 분리
X = df[["역번호", "hour", "평일주말", "상행", "승차", "승하차인원", "혼잡도"]].values
y = df["target"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [14]:
# 8. 딥러닝 모델 정의&학습
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=40,
    batch_size=64,
    verbose=2
)


c:\Users\user\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
146436/146436 - 178s - 1ms/step - loss: 137317.0781 - mae: 265.9102 - val_loss: 136621.5156 - val_mae: 265.3067
Epoch 2/40
146436/146436 - 178s - 1ms/step - loss: 136512.6094 - mae: 265.1358 - val_loss: 136622.4844 - val_mae: 264.5733
Epoch 3/40
146436/146436 - 178s - 1ms/step - loss: 136509.7812 - mae: 265.1390 - val_loss: 136607.4688 - val_mae: 264.7749
Epoch 4/40
146436/146436 - 178s - 1ms/step - loss: 136510.9062 - mae: 265.1371 - val_loss: 136605.1562 - val_mae: 264.9651
Epoch 5/40
146436/146436 - 178s - 1ms/step - loss: 136511.5625 - mae: 265.1411 - val_loss: 136638.9062 - val_mae: 264.4438
Epoch 6/40
146436/146436 - 179s - 1ms/step - loss: 136509.9375 - mae: 265.1359 - val_loss: 136606.7656 - val_mae: 264.7996
Epoch 7/40
146436/146436 - 178s - 1ms/step - loss: 136510.9375 - mae: 265.1374 - val_loss: 136629.4219 - val_mae: 264.5136
Epoch 8/40
146436/146436 - 178s - 1ms/step - loss: 136512.8438 - mae: 265.1368 - val_loss: 136630.2344 - val_mae: 265.4090
Epoch 9/40
14643

In [15]:
# 9. 예측 함수
def dl_predict(역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도):
    arr = np.array([[역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도]])
    arr_scaled = scaler.transform(arr)
    pred = model.predict(arr_scaled)
    return int(pred[0][0])


In [16]:
# 10. 예측 CLI
def user_predict():
    print("예시: 역번호 150 / 08시 / 평일=0, 주말=1 / 상행=1 하행=0 / 승차=1 하차=0 / 인원수 / 혼잡도(%)")
    역번호 = int(input("역번호: "))
    hour = int(input("시(hour): "))
    평일주말 = int(input("평일=0, 주말=1: "))
    상행 = int(input("상행=1, 하행=0: "))
    승차 = int(input("승차=1, 하차=0: "))
    승하차인원 = float(input("승하차 인원: "))
    혼잡도 = float(input("혼잡도(%): "))
    pred = dl_predict(역번호, hour, 평일주말, 상행, 승차, 승하차인원, 혼잡도)
    print(f"예측 인원: {pred} 명")

In [19]:
# 실행 예시
user_predict()

예시: 역번호 150 / 08시 / 평일=0, 주말=1 / 상행=1 하행=0 / 승차=1 하차=0 / 인원수 / 혼잡도(%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
예측 인원: 564 명
